In [1]:
import sys
sys.path.append('../src')
from db import *
from sqlalchemy import func
from sqlalchemy.orm import aliased
import numpy as np
from sqlalchemy import or_, and_
from tqdm import tqdm
from dask.distributed import Client

/Users/abuonomo/code/DataSquad/astro2020/venv/lib/python3.7/site-packages/tqdm/std.py:651: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [13]:
db_loc = '../scratch/test.sqlite'

In [14]:
engine = create_engine(f"sqlite:///{db_loc}")
Session = sessionmaker(bind=engine)
session = Session()

In [52]:
bh = (
    session.query(PaperKeywords.paper_bibcode, PaperKeywords.raw_keyword, PaperKeywords.score, Paper)
    .join(Paper)
    .filter(PaperKeywords.raw_keyword == 'black hole')
    .filter(PaperKeywords.score == None)
    .order_by(func.length(Paper.abstract))
    .first()
)

In [59]:
bh[-1].get_feature_text()

'Microscopic origin of the Bekenstein-Hawking entropy. The Bekenstein-Hawking area-entropy relation SBH = A/(4) is derived for a class of five-dimensional extremal black holes in string theory by counting the degeneracy of BPS soliton bound states. '

In [57]:
bh2 = (
    session.query(PaperKeywords.paper_bibcode, PaperKeywords.raw_keyword, PaperKeywords.score, Paper)
    .join(Paper)
    .filter(PaperKeywords.raw_keyword == 'black hole')
    .filter(PaperKeywords.score != None)
    .order_by(func.length(Paper.abstract))
    .first()
)

In [58]:
bh2[-].get_feature_text()

'Black Holes in Higher Dimensions. We review black-hole solutions of higher-dimensional vacuum gravity and higher-dimensional supergravity theories. The discussion of vacuum gravity is pedagogical, with detailed reviews of Myers-Perry solutions, black rings, and solution-generating techniques. We discuss black-hole solutions of maximal supergravity theories, including black holes in anti-de Sitter space. General results and open problems are discussed throughout. '

In [66]:
[(p.raw_keyword, p.score) for p in bh[-1].keywords]

[('state', 0.011182539771310823),
 ('lack', None),
 ('relation', None),
 ('class', 0.03279630223045727),
 ('black hole', None),
 ('origin', None),
 ('string theory', 0.06890874950642653),
 ('area', None),
 ('theory', None),
 ('degeneracy', 0.02655961718559421),
 ('five-dimensional extremal black hole', 0.13282932929223656),
 ('area-entropy relation SBH', 0.10100352236833343),
 ('BPS soliton', 0.03817872221387163),
 ('A/(4', 0.03301590089145787),
 ('microscopic origin', 0.01872708362609707)]

In [47]:
bh = (
    session.query(PaperKeywords.paper_bibcode, PaperKeywords.raw_keyword, PaperKeywords.score, Paper)
    .join(Paper)
    .filter(PaperKeywords.raw_keyword == 'black hole')
    .filter(PaperKeywords.score == None)
    .first()
)

In [46]:
bh

('2010MNRAS.406.2650M',
 'black hole',
 None,
 <Paper(bibcode="2010MNRAS.406.2650M", title="Electromagnetic counterparts of compact object mergers powered by the radioactive decay of r-process nuclei")>)

In [4]:
po = PaperOrganizer(no_below=10, no_above=0.5, min_mean_score=0.01, year_min=2000, year_max=2007, journal_blacklist=["arXiv"])

In [11]:
po._get_filtered_keywords(session, Keyword.id).first()

(14429)

In [5]:
# kwd_query = po._get_filtered_keywords(session)

# print(po.journal_blacklist)

# for j in po.journal_blacklist:
#     kwd_query = kwd_query.filter(~Paper.bibcode.contains(j))

# kwd_query[2]

In [6]:
corpus_size = session.query(Paper).count()
no_above_abs = int(po.no_above * corpus_size)
kwd_query = (
    session.query(
        Keyword, func.count(Keyword.id), func.avg(PaperKeywords.score)
    )
    .join(PaperKeywords)
    .join(Paper)
    .group_by(Keyword.id)
    .order_by(func.avg(PaperKeywords.score).desc())
    .having(func.count() >= po.no_below)
    .having(func.count() <= no_above_abs)
    .having(func.avg(PaperKeywords.score) >= po.min_mean_score)
)
for j in po.journal_blacklist:
    kwd_query = kwd_query.filter(~Paper.bibcode.contains(j))

In [7]:
kwds, _, _ = zip(*kwd_query.all())
kwd_ids = [k.id for k in kwds]

from scipy.sparse import coo_matrix
from gensim.matutils import Sparse2Corpus
from gensim.corpora import Dictionary

q = (
    session.query(PaperKeywords.paper_bibcode, PaperKeywords.keyword_id, PaperKeywords.count)
    .filter(PaperKeywords.keyword_id.in_(kwd_ids))
    .group_by(PaperKeywords.paper_bibcode)
    .order_by(PaperKeywords.paper_bibcode)
)

bibs, keyword_ids, counts = zip(*q)

corp_ind_to_bib = {i: b for i, b in enumerate(set(bibs))}
bib_to_corp_ind = {b: i for i, b in id_to_bib.items()}

dct_ind_to_kwd_ind = {i: k for i, k in enumerate(set(keyword_ids))}
kwd_id_to_dct_ind = {k: i for i, k in dct_ind_to_kwd_ind.items()}

corp_inds = [bib_to_corp_ind[b] for b in bibs]
dct_inds = [kwd_id_to_dct_ind[k] for k in keyword_ids]

coo_corpus =((b, k, c) for b, k, c in zip(corp_inds, dct_inds, counts))
a = np.fromiter(coo_corpus, dtype=[('row', int), ('col', int), ('value', int)])
m = coo_matrix((a['value'], (a['row'], a['col'])))
corpus = Sparse2Corpus(m)

q = session.query(Keyword.id, Keyword.keyword).filter(Keyword.id.in_(set(keyword_ids)))
id2word = {kwd_id_to_dct_ind[i]: k for i, k in q}
dct = Dictionary.from_corpus(corpus, id2word=id2word)

In [ ]:
corpus_size = session.query(Paper).count()
no_above_abs = int(po.no_above * corpus_size)
kwd_query = (
    session.query(
        Keyword, func.count(Keyword.id), func.avg(PaperKeywords.score)
    )
    .join(PaperKeywords)
    .join(Paper)
    .group_by(Keyword.id)
    .order_by(func.avg(PaperKeywords.score).desc())
    .having(func.count() >= po.no_below)
    .having(func.count() <= no_above_abs)
    .having(func.avg(PaperKeywords.score) >= po.min_mean_score)
)
for j in po.journal_blacklist:
    kwd_query = kwd_query.filter(~Paper.bibcode.contains(j))

In [ ]:
kwd_query.group_by

In [5]:
# topic_range = [10, 20, 30]

In [1]:
import dask.distributed

In [2]:
import logging

In [6]:
%%timeit -r 1 -n 1
models = po.make_all_topic_models(session, topic_range, alpha="auto", eta="auto", iterations=200, passes=5, eval_every=1)

distributed.dashboard.proxy - INFO - To route to workers diagnostics web server please install jupyter-server-proxy: python -m pip install jupyter-server-proxy
distributed.scheduler - INFO - Clear task state
distributed.scheduler - INFO -   Scheduler at:     tcp://127.0.0.1:52800
distributed.scheduler - INFO -   dashboard at:            127.0.0.1:8787
distributed.nanny - INFO -         Start Nanny at: 'tcp://127.0.0.1:52802'
distributed.nanny - INFO -         Start Nanny at: 'tcp://127.0.0.1:52803'
distributed.nanny - INFO -         Start Nanny at: 'tcp://127.0.0.1:52804'
distributed.nanny - INFO -         Start Nanny at: 'tcp://127.0.0.1:52805'
distributed.scheduler - INFO - Register worker <Worker 'tcp://127.0.0.1:52806', name: 0, memory: 0, processing: 0>
distributed.scheduler - INFO - Starting worker compute stream, tcp://127.0.0.1:52806
distributed.core - INFO - Starting established connection
distributed.scheduler - INFO - Register worker <Worker 'tcp://127.0.0.1:52808', name: 3,

54.6 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [7]:
client

NameError: name 'client' is not defined

distributed.scheduler - INFO - Register worker <Worker 'tcp://127.0.0.1:50872', name: tcp://127.0.0.1:50872, memory: 0, processing: 0>
distributed.scheduler - INFO - Starting worker compute stream, tcp://127.0.0.1:50872
distributed.core - INFO - Starting established connection
distributed.scheduler - INFO - Remove worker <Worker 'tcp://127.0.0.1:50872', name: tcp://127.0.0.1:50872, memory: 0, processing: 0>
distributed.core - INFO - Removing comms to tcp://127.0.0.1:50872


In [7]:
cohs = po.get_coherences(models)

n_topics=10:   0%|          | 0/3 [00:00<?, ?it/s]DEBUG:gensim.models.coherencemodel:Setting topics to those of the model: LdaModel(num_terms=1572, num_topics=10, decay=0.5, chunksize=2000)
INFO:gensim.topic_coherence.text_analysis:CorpusAccumulator accumulated stats from 1000 documents
INFO:gensim.topic_coherence.text_analysis:CorpusAccumulator accumulated stats from 2000 documents
INFO:gensim.topic_coherence.text_analysis:CorpusAccumulator accumulated stats from 3000 documents
INFO:gensim.topic_coherence.text_analysis:CorpusAccumulator accumulated stats from 4000 documents
INFO:gensim.topic_coherence.text_analysis:CorpusAccumulator accumulated stats from 5000 documents
INFO:gensim.topic_coherence.text_analysis:CorpusAccumulator accumulated stats from 6000 documents
INFO:gensim.topic_coherence.text_analysis:CorpusAccumulator accumulated stats from 7000 documents
n_topics=20:   0%|          | 0/3 [00:00<?, ?it/s]DEBUG:gensim.models.coherencemodel:Setting topics to those of the model: L

In [15]:
cohs

[-12.462846980899453, -15.21323348973292, -16.516400239083705]

In [6]:
tokens = po.get_tokens(session)

100%|██████████| 7245/7245 [00:14<00:00, 511.87it/s]


In [25]:
kwd_ids = [k.id for k, _, _ in kwds]

In [18]:
papers = session.query(Paper)

In [20]:
p = papers.first()

In [53]:
tokens0 = [[pk.keyword.keyword] * pk.count for pk in p.keywords] #if pk.keyword_id in kwd_ids]
tokens = [t for ts in tokens0 for t in ts]

In [54]:
tokens

['sky survey',
 'sky survey',
 'Wide-field Infrared Survey Explorer',
 'Wide-field Infrared Survey Explorer',
 'European Southern Observatory Schmidt',
 'point source sensitivity well',
 'Palomar Observatory Schmidt',
 'United Kingdom Schmidt',
 'high signal-to-noise source',
 'WISE',
 'WISE',
 'WISE',
 'WISE',
 'Initial On-orbit Performance',
 'InfraRed Astronomical Satellite',
 'denser coverage',
 'ecliptic pole',
 'useful tool',
 'second time',
 'Mission Description',
 'astrometric precision',
 'zodiacal background',
 'angular resolution',
 'unconfused region',
 'μm',
 'μm',
 'astronomy',
 'value',
 'decade',
 'Micron',
 'cryogen',
 'November',
 'launch',
 'December',
 'mJy',
 'July',
 'band',
 'January',
 'wavelength',
 '0farcs15']

In [25]:
# session.query(PaperKeywords).join(Paper).filter(PaperKeywords.raw_keyword == "gamma").all()

In [26]:
year_counts = session.query(Paper.year, func.count(Paper.year)).group_by(Paper.year).all()

session.query(Keyword).filter(Keyword.keyword == "black hole").first().get_years(session)

session.query(Paper.year, func.count(Paper.year)) \
    .join(PaperKeywords) \
    .join(Keyword) \
    .filter(Keyword.keyword.ilike('mars')) \
    .group_by(Paper.year) \
    .all()

session.query(Keyword).filter(Keyword.keyword.ilike('universe'))[0].papers[1].paper.get_feature_text()

!pip install snakeviz

%load_ext snakeviz

paper_kwd = session.query(PaperKeywords).join(Keyword).filter(Keyword.keyword=="measurement").first()

%%timeit -n 1 -r 1
added = 0
q = (
    session.query(Paper)
    .filter(~Paper.keywords.any(PaperKeywords.keyword_id == paper_kwd.keyword_id))
    .filter(
        or_(
            Paper.title.contains(paper_kwd.raw_keyword), 
            Paper.abstract.contains(paper_kwd.raw_keyword)
        )
    ).update
)
# %%timeit
# papers = q.all()
# with session.no_autoflush:
#     for p in papers:
#     #     with session.no_autoflush:
#         assoc = PaperKeywords(raw_keyword=paper_kwd.raw_keyword, score=np.nan)
#         assoc.keyword = paper_kwd.keyword
#         p.keywords.append(assoc)
#         added += 1
# session.rollback()

In [201]:
session.query(Keyword).count()

133320

In [237]:
session.rollback()

In [242]:
def get_pk_dict(p, paper_kwd):
    d = {
    'raw_keyword': paper_kwd.raw_keyword,
    'keyword_id': paper_kwd.keyword.id,
    'paper_bibcode': p.bibcode,
    }
    return d

In [11]:
session.query(PaperKeywords).group_by(PaperKeywords.raw_keyword).first()

<PaperKeywords(paper_bibcode="1997ApJ...486..665P", keyword.keyword="+")>

In [1]:
# %%timeit -n 1 -r 1
added = 0
t = 100
all_records = []
with session.no_autoflush:
    pbar = tqdm(session.query(PaperKeywords)[100:100+t], total=t)
    for paper_kwd in pbar:
        pbar.set_description(paper_kwd.raw_keyword)
        q = (
            session.query(Paper)
            .filter(~Paper.keywords.any(PaperKeywords.keyword_id == paper_kwd.keyword_id))
            .filter(
                or_(
                    Paper.title.contains(paper_kwd.raw_keyword), 
                    Paper.abstract.contains(paper_kwd.raw_keyword)
                )
            )
        )
        papers = q.all()
        for p in papers:
            d = get_pk_dict(p, paper_kwd)
            all_records.append(d)
            added += 1

NameError: name 'session' is not defined

In [255]:
len(all_records)

11830

In [256]:
%%timeit -n 1 -r 1
engine.execute(
    PaperKeywords.__table__.insert(),
    all_records,
)

280 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [18]:
session.query(PaperKeywords).join(Keyword).filter(Keyword.keyword == 'measurement').count()

1133

In [13]:
print(session.query(PaperKeywords).join(Keyword).filter(Keyword.keyword == 'measurement'))

SELECT paper_keywords.paper_bibcode AS paper_keywords_paper_bibcode, paper_keywords.keyword_id AS paper_keywords_keyword_id, paper_keywords.score AS paper_keywords_score, paper_keywords.raw_keyword AS paper_keywords_raw_keyword 
FROM paper_keywords JOIN keywords ON keywords.id = paper_keywords.keyword_id 
WHERE keywords.keyword = ?


In [222]:
engine

Engine(sqlite:///../scratch/test.sqlite)

In [216]:
session.flush()

In [150]:
session.query(PaperKeywords).join(Keyword).filter(Keyword.keyword == 'measurement').count()

307

In [148]:
added

15513

In [14]:
# session.rollback()
session.close()

In [71]:
q.first()

('2010MNRAS.409..531R')

In [52]:
KeywordAlias = aliased(Keyword)
q = (
 session.query(Keyword, func.count(Keyword.id))
    .join(PaperKeywords)
    .join(Paper)
    .group_by(Keyword.id)
#     .filter(Paper.year == 2010)
    .order_by(func.count(Keyword.id).desc())
    .having(func.count() > 10)
    .having(funct.count() < )
)

In [50]:
print(q)

SELECT keywords.id AS keywords_id, keywords.keyword AS keywords_keyword, count(keywords.id) AS count_1 
FROM keywords JOIN paper_keywords ON keywords.id = paper_keywords.keyword_id JOIN papers ON papers.bibcode = paper_keywords.paper_bibcode GROUP BY keywords.id 
HAVING count(*) > ? ORDER BY count(keywords.id) DESC


In [51]:
q.count()

3225

In [11]:
session.query(Paper).count()

7245

In [37]:
# q[0:100]

In [12]:
tokens = ([pk.keyword.keyword for pk in paper.keywords] for paper in session.query(Paper).all())

In [14]:
nlp = get_spacy_nlp()

In [22]:
pm = PaperMiner(nlp)

In [23]:
pm.get_dictionary(session)

In [32]:
corpus = pm.get_corpus(session)

In [70]:
session.rollback()

In [71]:
session.close()